In [1]:
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
# import matplotlib.pyplot as plt
import json
from datetime import datetime

In [2]:
print(tf.version.VERSION)

2.18.0-dev20240611


### Specify here the NN topology and the dataframe to be loaded

In [3]:
#specify the NN topology here:
#input_layer_nodes = number_of_attributes
number_of_attributes = 32
hidden_layer_nodes = 32
output_layer_nodes = 4


use_case = "stream-video"
dataset_folder = f"../datasets/nprint-raw/{use_case}/"
features_rankings = f"../datasets/nprint-raw/{use_case}/feature-importance.csv"



###Define the model

In [4]:
BATCH_SIZE = 512
EPOCHS = 500

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

###Import the data

In [5]:
df_train = pd.read_csv(f'{dataset_folder}X.csv')
df_test = pd.read_csv(f'{dataset_folder}X_val.csv')
label_train = pd.read_csv(f'{dataset_folder}y.csv')
label_test = pd.read_csv(f'{dataset_folder}y_val.csv')

df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)

# display(df_train)
# display(df_test)
# display(label_train)
display(label_test)

,label
0,1
1,3
2,3
3,1
4,2
...,...
728,1
729,3
730,2
731,3


In [6]:
#select a sub-dataframe with only the top attrs rate by the AUTOGLUON feature importance algorithm
#load the file with the rankings
features_rankings_df = pd.read_csv(features_rankings)
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

#make a list with only the best features
feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:number_of_attributes]
# print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]
# display(df_train)
# display(df_test)

In [7]:
#RTS values conversion for video-streaming use-case
mini = []
maxi = []
column_list = ["pkt_1_rts", "pkt_2_rts", "pkt_3_rts", "pkt_4_rts", "pkt_5_rts", "pkt_6_rts", "pkt_7_rts", "pkt_8_rts"]
for column_name in column_list:
    mini.append(df_test[column_name].min())
    print(mini)    
    maxi.append(df_test[column_name].max())
    print(maxi)

maximo = max(maxi)
minimo = min(mini)
print(maximo)


display(df_test)
for column_name in column_list:
    aux = []
    for x in df_test[column_name]:
        normalized_value = (x - minimo) / (maximo - minimo)
        scaled_value = normalized_value * (2**8 - 1)
        aux.append(scaled_value)
    df_test[column_name]=aux    
display(df_test)

mini = []
maxi = []
for column_name in column_list:
    mini.append(df_train[column_name].min())
    print(mini)    
    maxi.append(df_train[column_name].max())
    print(maxi)

maximo = max(maxi)
minimo = min(mini)
print(maximo)


display(df_train)
for column_name in column_list:
    aux = []
    for x in df_train[column_name]:
        normalized_value = (x - minimo) / (maximo - minimo)
        scaled_value = normalized_value * (2**8 - 1)
        aux.append(scaled_value)
    df_train[column_name]=aux
    
display(df_train)

[0]
[358648184]
[0, 0]
[358648184, 356388935]
[0, 0, -1]
[358648184, 356388935, 358855971]
[0, 0, -1, -1]
[358648184, 356388935, 358855971, 361459473]
[0, 0, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967]
[0, 0, -1, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967, 357808985]
[0, 0, -1, -1, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967, 357808985, 361397597]
[0, 0, -1, -1, -1, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967, 357808985, 361397597, 364156899]
364156899


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,1,151,1,0,53,1431882,0,0,...,0,370,0,0,0,0,0,314,1,0
1,1,0,0,2076744,0,0,3786926,119549565,1,0,...,1,1197,0,0,0,1,0,31323367,0,0
2,1,0,0,967187,0,0,150,196629,0,1,...,1,7963,0,0,1,1,0,58844,0,0
3,1,0,1,94780,1,0,319,274,1,1,...,0,272,0,0,0,0,0,704,1,0
4,1,0,1,40,1,0,7267,36,1,1,...,1,72,1,0,1,0,1,35,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0,1,0,421237,0,0,47848,121529,0,0,...,1,119321,1,0,0,1,0,183,0,0
729,1,1,0,72,0,0,3044,12488,0,1,...,1,3547,0,0,0,1,0,56,1,0
730,1,0,0,1069169,1,1,170952,104,0,0,...,0,284,0,1,0,1,0,73,0,1
731,1,0,1,148,0,0,1178443,1453628,0,0,...,1,167,0,0,0,1,0,277150,1,0


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,1,0.000106,1,0,0.000038,1.002673,0,0,...,0,0.000260,0,0,0,0,0,0.000221,1,0
1,1,0,0,1.454236,0,0,2.651787,83.714298,1,0,...,1,0.000839,0,0,0,1,0,21.934114,0,0
2,1,0,0,0.677271,0,0,0.000106,0.137690,0,1,...,1,0.005577,0,0,1,1,0,0.041206,0,0
3,1,0,1,0.066370,1,0,0.000224,0.000193,1,1,...,0,0.000191,0,0,0,0,0,0.000494,1,0
4,1,0,1,0.000029,1,0,0.005089,0.000026,1,1,...,1,0.000051,1,0,1,0,1,0.000025,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0,1,0,0.294971,0,0,0.033506,0.085101,0,0,...,1,0.083555,1,0,0,1,0,0.000129,0,0
729,1,1,0,0.000051,0,0,0.002132,0.008745,0,1,...,1,0.002484,0,0,0,1,0,0.000040,1,0
730,1,0,0,0.748684,1,1,0.119709,0.000074,0,0,...,0,0.000200,0,1,0,1,0,0.000052,0,1
731,1,0,1,0.000104,0,0,0.825203,1.017900,0,0,...,1,0.000118,0,0,0,1,0,0.194074,1,0


[0]
[359235600]
[0, 0]
[359235600, 359517995]
[0, 0, -1]
[359235600, 359517995, 359397735]
[0, 0, -1, -1]
[359235600, 359517995, 359397735, 358981693]
[0, 0, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870]
[0, 0, -1, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870, 363893498]
[0, 0, -1, -1, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870, 363893498, 361459333]
[0, 0, -1, -1, -1, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870, 363893498, 361459333, 362452485]
363893498


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,0,1362319,0,0,71,132,0,0,...,1,296,0,0,0,1,0,69,0,0
1,1,0,1,285,1,0,926,104160,0,1,...,0,4390256,0,0,0,0,0,634,0,0
2,1,0,1,326,0,0,219,205,1,1,...,0,1352462,0,1,0,1,0,121,1,0
3,1,0,0,33,0,1,14186,20,0,1,...,1,43,1,1,0,0,1,38,1,1
4,1,1,0,1190046,0,1,59,116,1,1,...,1,244,0,1,0,1,1,55,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,1,0,0,241610,1,1,7998,621382,0,1,...,1,11881,0,0,1,1,0,44405,1,0
6593,1,0,1,214,1,0,142,1531520,0,0,...,0,320,0,0,0,0,0,78,0,0
6594,1,0,1,155,1,1,445725,1395299,1,0,...,1,162,0,0,0,1,0,505802,1,0
6595,1,0,1,27,1,0,24,24,0,1,...,0,150058,0,0,0,0,0,20,1,0


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,0,0.954652,0,0,0.000050,0.000093,0,0,...,1,0.000208,0,0,0,1,0,0.000049,0,0
1,1,0,1,0.000200,1,0,0.000650,0.072991,0,1,...,0,3.076492,0,0,0,0,0,0.000445,0,0
2,1,0,1,0.000229,0,0,0.000154,0.000144,1,1,...,0,0.947745,0,1,0,1,0,0.000085,1,0
3,1,0,0,0.000024,0,1,0.009942,0.000015,0,1,...,1,0.000031,1,1,0,0,1,0.000027,1,1
4,1,1,0,0.833931,0,1,0.000042,0.000082,1,1,...,1,0.000172,0,1,0,1,1,0.000039,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,1,0,0,0.169310,1,1,0.005605,0.435437,0,1,...,1,0.008326,0,0,1,1,0,0.031118,1,0
6593,1,0,1,0.000151,1,0,0.000100,1.073220,0,0,...,0,0.000225,0,0,0,0,0,0.000055,0,0
6594,1,0,1,0.000109,1,1,0.312344,0.977763,1,0,...,1,0.000114,0,0,0,1,0,0.354444,1,0
6595,1,0,1,0.000020,1,0,0.000018,0.000018,0,1,...,0,0.105155,0,0,0,0,0,0.000015,1,0


### NN training

In [8]:
# Begin NN training
NUMBER_OF_ATTRS = df_train.shape[1]

# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Normalize the data
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

# Execute Trainning 
print('Starting training for',number_of_attributes,'attributes')
model = get_basic_model()
model.fit(df_train, label_train, epochs=EPOCHS, verbose=2, batch_size=BATCH_SIZE)

# and Testing
test_loss, one_test_acc =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
tf_predictions_probabilities = model.predict(df_test)

#Crate Confusion Matrix for better understanding of results
tf_predictions = []
for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)
  j_max = x.argmax()
  tf_predictions.append(j_max)

conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
print(conf_m)

Starting training for 32 attributes
Epoch 1/500
13/13 - 0s - 37ms/step - accuracy: 0.2665 - loss: 1.5470
Epoch 2/500
13/13 - 0s - 2ms/step - accuracy: 0.4525 - loss: 1.2416
Epoch 3/500
13/13 - 0s - 2ms/step - accuracy: 0.6551 - loss: 1.0098
Epoch 4/500
13/13 - 0s - 2ms/step - accuracy: 0.7494 - loss: 0.8468
Epoch 5/500
13/13 - 0s - 2ms/step - accuracy: 0.7949 - loss: 0.7265
Epoch 6/500
13/13 - 0s - 2ms/step - accuracy: 0.8333 - loss: 0.6330
Epoch 7/500
13/13 - 0s - 2ms/step - accuracy: 0.8602 - loss: 0.5572
Epoch 8/500
13/13 - 0s - 2ms/step - accuracy: 0.8781 - loss: 0.4962
Epoch 9/500
13/13 - 0s - 2ms/step - accuracy: 0.8856 - loss: 0.4470
Epoch 10/500
13/13 - 0s - 2ms/step - accuracy: 0.8922 - loss: 0.4075
Epoch 11/500
13/13 - 0s - 2ms/step - accuracy: 0.8966 - loss: 0.3755
Epoch 12/500
13/13 - 0s - 2ms/step - accuracy: 0.8990 - loss: 0.3492
Epoch 13/500
13/13 - 0s - 2ms/step - accuracy: 0.9001 - loss: 0.3279
Epoch 14/500
13/13 - 0s - 2ms/step - accuracy: 0.9053 - loss: 0.3099
Epoch 

In [9]:
# Model info
model.summary()

norm_layer_weights = model.layers[0].get_weights()[0] #weight
norm_layer_biases  = model.layers[0].get_weights()[1] #bias
hidden_layer_weights = model.layers[1].get_weights()[0] #weight
hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
out_layer_weights = model.layers[2].get_weights()[0] #weight
out_layer_biases  = model.layers[2].get_weights()[1] #bias
print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
print('\nHidden -> Output Layer bias:\n',out_layer_biases)
print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ normalization (Normalization)        │ (None, 32)                  │              65 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,631 (14.19 KB)

 Trainable params: 1,188 (4.64 KB)

 Non-trainable params: 65 (264.00 B)

 Optimizer params: 2,378 (9.29 KB)


Input -> Normalization Layer bias:
 [3.2496046e-02 2.1035653e-01 2.6068407e-01 8.7901993e+01 2.6656783e-01
 2.4136128e-01 2.2219403e+02 1.4282243e+02 2.5592527e-01 2.3592572e-01
 2.5280026e-01 3.0108793e-02 4.6279755e+02 2.5586393e-01 2.3382136e-01
 2.3119856e-01 2.7180219e-01 4.3021112e+02 1.7713346e-01 2.4548876e-01
 2.1863680e-01 4.9189352e+02 2.3726189e-01 1.9919359e+01 2.2585610e-01
 2.5386244e-01 1.5285288e-01 2.1156773e-01 2.2041728e-01 5.5625623e+02
 2.4989355e-01 1.9132951e-01]

Input -> Normalization Layer weights:
 [0.96831894 0.30089435 0.3944217  0.9774101  0.42746702 0.32545096
 1.8325263  1.2299721  0.5115962  0.6342277  0.53372747 0.03107473
 3.3500943  0.62634534 0.6352888  0.6371078  0.50477487 3.3187194
 0.7738366  0.63362134 0.26936486 3.3010285  0.3628922  0.2995473
 0.28558436 0.45308474 0.15507048 0.7013794  0.31120205 3.817457
 0.48961648 0.25405487]

Normalization -> Hidden Layer bias:
 [ 0.43173155  0.04158311  0.2371507   0.04819361  0.24129122 -0.2554991
  

In [10]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "number_of_attributes": number_of_attributes,
    "hidden_layer_nodes": hidden_layer_nodes,
    "output_layer_nodes": output_layer_nodes,
    "accuracy_test": one_test_acc,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS
    #"number_of_samples": number_of_samples,
    #"df_number": df_number
}

26-07-2024-16-09-54


In [11]:
#TO DO: use save_models instead of save_weights

# Model parameters json file generation, create file with date-time string to prevent unwated/accidental overwrites
os.chdir('../tf-params-reports/')
print(os.getcwd())

title_parameters_save = f"nn-nprint-{use_case}-model-parameters-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-{use_case}-model-parameters-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

# Model file with weights and other params
os.chdir('../tf-models/') 
print(os.getcwd())

title_model_save = f'nn-nprint-{use_case}-model-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.keras'
model.save(title_model_save)

# Model weights file generation
# os.chdir('../tf-model-weights/')
# print(os.getcwd())

# title_model_save = f'nn-nprint-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.weights.h5'
# model.save_weights(title_model_save) # Calling `save('my_model')` creates a SavedModel folder `my_model`.

# title_model_save = f'nn-nprint-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.weights.h5'
# model.save_weights(title_model_save) # Calling `save('my_model')` creates a SavedModel folder `my_model`.
    
# model.load_weights(f'nn-nprint-app-iden-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.weights.h5')

C:\NN-P4\tf-params-reports
C:\NN-P4\tf-models


In [12]:
# Acervo - comandos úteis os python
# %pwd
# os.chdir(C:\NN-P4\nn-reports)
# cwd = os.getcwd()
# print(cwd)
# os.listdir()
# os.chdir('../nn-reports/')

In [13]:
# df_train = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/X.pkl')
# df_test = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/X_val.pkl')
# label_train = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/y.pkl')
# label_test = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/y_val.pkl')